In [ ]:
IMDB_MOVIES_PATH = 'https://datasets.imdbws.com/title.basics.tsv.gz'

In [ ]:
import boto3
import json
import imdb2tmdb
import pandas as pd

In [ ]:
sqs = boto3.resource('sqs')
queue = sqs.get_queue_by_name(QueueName='hudsonmendes-imdb2tmdb-movies-download-queue')

In [ ]:
df = pd.read_csv(IMDB_MOVIES_PATH, delimiter='\t', header=0)
df['initial'] = df['primaryTitle'].map(imdb2tmdb.imdb.get_initial_from)
initials = sorted(set([ initial for initial in df['initial'] if initial ]))
years = sorted(set([ year for year in df['startYear'] if year ]))

In [ ]:
for year, initial in zip(years, initials):
    print((year, initial))
    message = { 'year': year, 'initial': initial }
    body = json.dumps(message)
    queue.send_message(MessageBody=body)